In [1]:
#import sys
import numpy as np
#import pickle

import sqlite3 as sq

import re

from nltk.stem.snowball import SnowballStemmer
#from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [2]:
def load_data(path):
    dbname = path
    data = { 'text':[],'tag':[] }
    conn = sq.connect(dbname)
    try:
        c = conn.cursor()
        for row in c.execute("SELECT * FROM data"):
            data['text'] += [row[1]]
            data['tag'] += [row[2]]
    finally:
        conn.close()
    return data

In [3]:
def text_cleaner(text):
    text = text.lower() # приведение в lowercase,
    
    text = re.sub( r'https?://[\S]+', ' url ', text) # замена интернет ссылок
    text = re.sub( r'[\w\./]+\.[a-z]+', ' url ', text) 
 
    # text = re.sub( r'\d+[-/\.]\d+[-/\.]\d+', ' date ', text) # замена даты и времени
    # text = re.sub( r'\d+ ?гг?', ' date ', text) 
    # text = re.sub( r'\d+:\d+(:\d+)?', ' time ', text) 

    # text = re.sub( r'@\w+', ' tname ', text ) # замена имён twiter
    # text = re.sub( r'#\w+', ' htag ', text ) # замена хештегов

    text = re.sub( r'<[^>]*>', ' ', text) # удаление html тагов
    text = re.sub( r'[\W]+', ' ', text ) # удаление лишних символов

    stemmer = SnowballStemmer("russian")
    singles = [stemmer.stem(word) for word in text.split()]
    text = ' '.join(singles)

    # stw = ['в', 'по', 'на', 'из', 'и', 'или', 'не', 'но', 'за', 'над', 'под', 'то',
    #        'a', 'at', 'on', 'of', 'and', 'or', 'in', 'for', 'at' ]
    # remove = r'\b('+'|'.join(stw)+')\b'
    # text = re.sub(remove,' ', text)
    
    # text = re.sub( r'\b\w\b', ' ', text ) # удаление отдельно стоящих букв

    text = re.sub( r'\b\d+\b', ' digit ', text ) # замена цифр 

    return  text

In [4]:
# - - - - - - - - - - - - - - - - - - - - - - - - -
# разделение набора текстов (data)
# на тестовый и учебный наборы
# случайным образом в заданной пропорции (validation_split)
#
def train_test_split( data, validation_split = 0.2):
    sz = len(data['text'])
    indices = np.arange(sz)
    np.random.shuffle(indices)

    X = [ data['text'][i] for i in indices ]
    Y = [ data['tag'][i] for i in indices ]
    nb_validation_samples = int( validation_split * sz )

    return { 
        'train': { 'x': X[:-nb_validation_samples], 'y': Y[:-nb_validation_samples]  },
        'test': { 'x': X[-nb_validation_samples:], 'y': Y[-nb_validation_samples:]  }
    }

In [5]:
print("[i] загружаем данные...")
data = load_data('rss-all.sqlite')
data
print("\tсчитано: ", len(data['text']))
print("\tколичество исходных категорий:",len(set(data['tag'])))

# print("[i] очистка данных...")
# data['text'] = [ text_cleaner(t) for t in data['text'] ]
# print("[i] готово.")

print("[i] разделение данных...")
D = train_test_split( data )
print("[i] готово.")

[i] загружаем данные...
	считано:  3196
	количество исходных категорий: 13
[i] разделение данных...
[i] готово.


In [6]:
data

{'text': ['В Саудовской Аравии сняли первый антитеррористический мультфильм -\nтрехминутную ленту "Внимание!". "Внимание!" отражает точку зрения мирового\nсообщества на войну, развязанную терроризмом, и поэтому понятен всем",\n- заявил режиссер короткометражки Акрам Ага. Антитеррористический\nмультфильм уже демонстрировался на кинофестивалях в Швейцарии и Германии.\nАрабские зрители смогут увидеть его на 2-м ежегодном кинофестивале в\nДубае, который пройдет с 11 по 17 декабря.',
  'Вчера вечером в Японии состоялась премьера голливудского фильма о гейшах,\nвызвавшая негодование в связи с тем, что эти девушки представлены\nпроститутками, а играющие их актрисы - китаянки. Мало того, что главные\nроли фильма "Мемуары гейши" исполнили китайские актрисы, он еще и снят\nв Калифорнии, а не в Киото - традиционном месте обитания гейш - американским\nрежиссером по книге, написанной американцем. Фильм, показ которого\nпроисходил в здании стадиона для сумо, носит откровенно сексуальный\nхарактер и 

In [6]:
print("[i] печать данных...")

print("\tразмер тестовой выборки:", len(D['test'] ['y']))
print("\tразмер тренировочной выборки:", len(D['train'] ['y']))
print(D['test'] ['y'])

[i] печать данных...
	размер тестовой выборки: 639
	размер тренировочной выборки: 2557
['culture', 'politics', 'health', 'culture', 'tech', 'politics', 'incident', 'economics', 'economics', 'economics', 'sport', 'auto', 'incident', 'tech', 'culture', 'culture', 'tech', 'social', 'incident', 'sport', 'politics', 'tech', 'auto', 'science', 'tech', 'sport', 'auto', 'realty', 'sport', 'economics', 'culture', 'realty', 'politics', 'auto', 'health', 'science', 'politics', 'auto', 'incident', 'culture', 'health', 'tech', 'tech', 'science', 'politics', 'auto', 'sport', 'politics', 'incident', 'culture', 'incident', 'economics', 'culture', 'incident', 'economics', 'incident', 'politics', 'incident', 'culture', 'auto', 'social', 'incident', 'economics', 'sport', 'sport', 'economics', 'sport', 'economics', 'social', 'sport', 'sport', 'culture', 'tech', 'politics', 'science', 'tech', 'tech', 'tech', 'culture', 'science', 'politics', 'economics', 'politics', 'culture', 'incident', 'economics', 'sci

In [7]:
print("[i] обучение классификатора...")

    # text_clf = Pipeline([
    #                ('hashvect', HashingVectorizer() ),
    #                ('tfidf', TfidfTransformer(use_idf=False )),
    #                ('clf', SGDClassifier(loss='hinge')),
    #                ])
    #
    # text_clf = Pipeline([
    #                ('covect', CountVectorizer() ),
    #                ('tfidf', TfidfTransformer(preprocessor=text_cleaner,use_idf=False )),
    #                ('clf', SGDClassifier(loss='hinge')),
    #                ])


text_clf = Pipeline([
                ('tfidf', TfidfVectorizer()),
                ('clf', SGDClassifier(loss='hinge')),
                ])

text_clf.fit(D['train']['x'], D['train']['y'])

    
print("[i] тестируем...")

predicted = text_clf.predict( D['train']['x'] )
print("\taccuracy train: ", accuracy_score(  D['train']['y'] , predicted) )
    
predicted = text_clf.predict( D['test']['x'] )
print("\taccuracy test: ", accuracy_score(  D['test']['y'] , predicted) )

[i] обучение классификатора...


C:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


[i] тестируем...
	accuracy train:  0.9988267500977708
	accuracy test:  0.8904538341158059
